<a href="https://colab.research.google.com/github/aadityane93/Machine_Learning_Notebooks/blob/main/Sorting_line_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1BZJfMkmr5uyyL1iZGPQen-B3VTYrJjD0'
# folder_id = '1bxp8LJq3826UeOR44M9ZiDFX61Dp-xSf'
folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        new_file_name = f"{folder_title}-{index}"
        print(f"Downloading {file_title} as {new_file_name}")
        file.GetContentFile(new_file_name)

Processing folder: niO
Processing folder: biO
Processing folder: riO
Processing folder: wiO


In [5]:
!pip install tsfresh getml --upgrade
!pip install tsfresh

from google.colab import files
import numpy as np
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb
import getml
from getml.engine import launch
from getml.feature_learning import Relboost
from getml.predictors import LinearRegression, LogisticRegression  # Use predictors instead of models

# Launch the getml engine
launch()

Launching ./getML --allow-push-notifications=true --allow-remote-ips=false --home-directory=/root/.getML --in-memory=true --install=false --launch-browser=true --log=false --project-directory=/root/.getML/projects in /usr/local/lib/python3.11/dist-packages/getml/.getML/getml-community-1.5.0-amd64-linux...
Launched the getML Engine. The log output will be stored in /root/.getML/logs/getml_20250309053530.log


In [7]:
def read_file(filename):
    return pd.read_csv((filename), sep=',')
########################################################## INDEX ################################################

#Index to seperarate the read data into train and test dataframes
# index = index - 4
index_train = 50
index_test = 50

# Dictonaries to read the files
white_train = white_test = {}
blue_train = blue_test = {}
red_train = red_test = {}
nio_train = nio_test = {}

#Assigning df
white_train_df = white_test_df = pd.DataFrame()
blue_train_df = blue_test_df = pd.DataFrame()
red_train_df = red_test_df = pd.DataFrame()
nio_train_df = nio_test_df = pd.DataFrame()

#Reading files for training
for j in range (index_train):
  white_train[j] = read_file(f"wiO-{j}")
  blue_train[j] = read_file(f"biO-{j}")
  red_train[j] = read_file(f"riO-{j}")
  nio_train[j] = read_file(f"niO-{j}")
# print (index_train)

#Reading files for testing
for i in range (index_test):
  white_test[i] = read_file(f"wiO-{i+index_train}")
  blue_test[i] = read_file(f"biO-{i+index_train}")
  red_test[i] = read_file(f"riO-{i+index_train}")
  nio_test[i] = read_file(f"niO-{i+index_train}")
# print (i+index_train)

In [8]:
# Color df for Train Dataset
df_color_train = pd.DataFrame()
for i in range(len(white_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(0)])
for i in range(len(blue_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(1)])
for i in range(len(red_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(2)])
for i in range(len(nio_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(3)])
df_color_train = df_color_train.astype(int)
df_color_train.columns = ['color']


# Color df for Test Dataset
df_color_test = pd.DataFrame()
for i in range(len(white_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(0)])
for i in range(len(blue_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(1)])
for i in range(len(red_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(2)])
for i in range(len(nio_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(3)])
df_color_test = df_color_test.astype(int)
df_color_test.columns = ['color']

In [9]:
print(df_color_train.shape)
print(df_color_test.shape)

(200, 1)
(200, 1)


In [10]:
# Merging df of different Colors into a single df for Train Dataset

q= 0
r= index_train
s = index_train*2
t = index_train*3


for j in range(index_train):
  white_train[j] = pd.DataFrame(white_train[j])
  white_train[j]['fileno'] = q
  white_train_df = pd.concat([white_train_df, white_train[j]])

  blue_train[j] = pd.DataFrame(blue_train[j])
  blue_train[j]['fileno'] = r
  blue_train_df = pd.concat([blue_train_df, blue_train[j]])

  red_train[j] = pd.DataFrame(red_train[j])
  red_train[j]['fileno'] = s
  red_train_df = pd.concat([red_train_df, red_train[j]])

  nio_train[j] = pd.DataFrame(nio_train[j])
  nio_train[j]['fileno'] = t
  nio_train_df = pd.concat([nio_train_df, nio_train[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1



# Merging df of different Colors into a single df for Test Dataset

q= 0
r= index_test
s = index_test*2
t = index_test*3


for j in range(index_test):
  white_test[j] = pd.DataFrame(white_test[j])
  white_test[j]['fileno'] = q
  white_test_df = pd.concat([white_test_df, white_test[j]])

  blue_test[j] = pd.DataFrame(blue_test[j])
  blue_test[j]['fileno'] = r
  blue_test_df = pd.concat([blue_test_df, blue_test[j]])

  red_test[j] = pd.DataFrame(red_test[j])
  red_test[j]['fileno'] = s
  red_test_df = pd.concat([red_test_df, red_test[j]])

  nio_test[j] = pd.DataFrame(nio_test[j])
  nio_test[j]['fileno'] = t
  nio_test_df = pd.concat([nio_test_df, nio_test[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1


In [11]:
# for i in range (len(white_df)):
#   white_df['fileno'] = i
# for j in range (i, i + len(blue_df)):
#   blue_df['fileno'].append(j)
# for k in range (j,j+len(red_df)):
#   red_df['fileno'].append(j)
# for l in range (k, k+len(nio_df)):
#   nio_df['fileno'].append(j)

In [12]:
# white_df ['color'] = 0
# blue_df ['color'] = 1
# red_df ['color'] = 2
# nio_df ['color'] = 3

In [13]:
df_train = pd.concat([white_train_df, blue_train_df, red_train_df, nio_train_df])
df_test = pd.concat([white_test_df, blue_test_df, red_test_df, nio_test_df])


df_train = df_train.drop(columns=['Datum'])
df_test = df_test.drop(columns=['Datum'])
x_train_df = df_train.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])
x_test_df = df_test.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])
# y = df['color']
# j=0
# y_aggregated = pd.DataFrame()
# for i in range(len(df['fileno'])):
#   if df['fileno'].values[i] ==



  # if df['color'].values[i] == j:
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # elif df['color'].values[i] != j:
  #   j = j + 1
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # else:
  #   print('error')


In [14]:
df_train.tail()

,Uhrzeit,I_In,V_In,P_In(W),I_Out(A),V_Out(V),P_Out(W),Temp(°C),Energie_In(Wh),Energie_Out(Wh),fileno
863,2024-05-02 12:51:02.302558:302,0.31,11.70,3.6270,0.29,9.03,2.6187,37.88,0.027290,0.020563,199
864,2024-05-02 12:51:02.310972:310,0.25,11.61,2.9025,0.31,9.06,2.8086,37.88,0.027297,0.020570,199
865,2024-05-02 12:51:02.323006:323,0.25,11.70,2.9250,0.25,9.03,2.2575,37.88,0.027307,0.020578,199
866,2024-05-02 12:51:02.331326:331,0.31,11.70,3.6270,0.27,9.00,2.4300,37.88,0.027315,0.020583,199
867,2024-05-02 12:51:02.343540:343,0.31,11.61,3.5991,0.31,9.03,2.7993,38.13,0.027327,0.020593,199


In [15]:
df_train['fileno'].values

array([  0,   0,   0, ..., 199, 199, 199])

In [ ]:
features_train =  extract_features(
    x_train_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

In [ ]:
# feature_file = f'features-{index*4}.csv'
# features.to_csv(feature_file, sep=',')
# files.download(feature_file)

In [ ]:
features_train.head()

In [ ]:
print(features_train.shape)
y_train_df= df_color_train['color']
y_train_df = y_train_df.reset_index(drop=True)
print(y_train_df.shape)

In [ ]:
y_train_df

In [ ]:
selected_features = select_features(features_train, y_train_df)

In [ ]:
selected_features.head()

In [ ]:
wex_train,x_test,y_train,y_test = train_test_split(selected_features,y, test_size=0.5, random_state=42, stratify=y)

In [ ]:
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
def xgb_(X, y, num_folds=5):

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=24)

    f1_scores = []
    accuracies = []
    precisions = []

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold_idx + 1}:")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Initialize and train the XGBoost model with regularization
        xgb1 = xgb.XGBClassifier(
            objective="multi:softmax",
            random_state=42,
            alpha=0.1,
            reg_lambda=0.1,
            max_depth=4,
            min_child_weight=10,
            subsample=0.8,
            colsample_bytree=0.8
        )

        # Fit the model
        xgb1.fit(X_train.to_numpy(), y_train.to_numpy().ravel())

        # Make predictions
        y_pred = xgb1.predict(X_test.to_numpy())

        # Ensure y_test is 1D for comparison
        y_test = y_test.to_numpy().ravel().astype(int)
        y_pred = y_pred.astype(int)

        # Calculate and store F1-score for this fold
        f1 = f1_score(y_test, y_pred, average='weighted')
        f1_scores.append(f1)

        # Calculate and store accuracy and precision for this fold
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        precision = precision_score(y_test, y_pred, average='weighted')
        precisions.append(precision)

        print(classification_report(y_test, y_pred))
        print(f"Fold {fold_idx + 1} F1-score: {f1:.4f}")
        print(f"Fold {fold_idx + 1} Accuracy: {accuracy:.4f}")
        print(f"Fold {fold_idx + 1} Precision: {precision:.4f}")

        # Calculate and print confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print("Confusion Matrix:")
        print(cm)

        print("-" * 30)

    # Calculate and print average metrics across all folds
    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)

    print(f"Average F1-score across {num_folds} folds: {avg_f1:.4f}")
    print(f"Average Accuracy across {num_folds} folds: {avg_accuracy:.4f}")
    print(f"Average Precision across {num_folds} folds: {avg_precision:.4f}")

    # Final model: Train on the entire dataset
    print("Training final model on the entire dataset...")
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y, test_size=0.2, random_state=42)

    # Final model with regularization
    final_model = xgb.XGBClassifier(
        objective="multi:softmax",
        random_state=42,
        alpha=0.1,
        reg_lambda=0.1,
        max_depth=5,
        min_child_weight=10,
        subsample=0.8,
        colsample_bytree=0.8
    )

    # Train on the entire dataset
    final_model.fit(X_train_final.to_numpy(), y_train_final.to_numpy().ravel())

    # Evaluate the final model on the test set
    y_pred_final = final_model.predict(X_test_final.to_numpy())

    # Calculate accuracy, precision, confusion matrix for final model
    final_accuracy = accuracy_score(y_test_final, y_pred_final)
    final_precision = precision_score(y_test_final, y_pred_final, average='weighted')
    final_cm = confusion_matrix(y_test_final, y_pred_final)

    print(f"Final Model Accuracy: {final_accuracy:.4f}")
    print(f"Final Model Precision: {final_precision:.4f}")
    print("Final Model Confusion Matrix:")
    print(final_cm)

    # Save the final model
    # with open('XGB_final_model.pkl', 'wb') as f:
    #     pickle.dump(final_model, f)
    # files.download('XGB_final_model.pkl')



In [ ]:
xgb_ (selected_features, y_train_df)

In [ ]:
features_test = extract_features(
    x_test_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

In [ ]:
#Using the same columns from the training data
features_test = features_test[selected_features.columns]

In [ ]:
print(features_test.shape)
y_test_df= df_color_test['color']
y_test_df = y_test_df.reset_index(drop=True)
print(y_test_df.shape)

In [ ]:
# model = RandomForestClassifier()
# model.fit(features_test, y_test)
y_pred = model.predict(features_test)
accuracy = accuracy_score(y_test_df, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test_df, y_pred))
print(confusion_matrix(y_test_df, y_pred))

In [ ]:
# xgb_ (features_test, y_test_df)

In [ ]:
# Read file into DataFrame

wio15_df = pd.DataFrame(pd.read_csv(('wiO-15'), sep=',', header=0))



wio15_df['fileno'] = 15
wio15_df = wio15_df.drop(columns=['Datum','P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])

extracted_features = extract_features(
    wio15_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)
wio_features = extracted_features[selected_features.columns]
y_pred = model.predict(wio_features)
print(y_pred)
accuracy = accuracy_score(pd.Series([0]), y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(pd.Series([0]), y_pred))
print(confusion_matrix(pd.Series([0]), y_pred))


In [26]:
x_train_df.head()

,Uhrzeit,I_In,V_In,I_Out(A),V_Out(V),Temp(°C),fileno
0,2024-05-02 11:28:32.754366:754,0.25,11.65,0.31,9.03,37.38,0
1,2024-05-02 11:28:32.766506:766,0.27,11.70,0.27,9.00,37.38,0
2,2024-05-02 11:28:32.774671:774,0.44,11.74,0.27,9.00,37.38,0
3,2024-05-02 11:28:32.786630:786,0.31,11.61,0.41,9.00,37.38,0
4,2024-05-02 11:28:32.794863:794,0.31,11.70,0.33,9.00,37.63,0


In [34]:
import pandas as pd
from getml.data import DataFrame, roles
from getml.engine import set_project, launch

set_project("my_project")

launch()

if not isinstance(x_train_df, pd.DataFrame):
    x_train_df = pd.DataFrame(x_train_df)

x_train_df["id"] = x_train_df.index.astype(str)

population = DataFrame.from_pandas(x_train_df, name="population")
population.set_role("id", roles.join_key)
population.set_role(["I_In", "V_In", "I_Out(A)", "V_Out(V)", "Temp(°C)"], roles.numerical)
population.set_role("Uhrzeit", roles.time_stamp)
population.set_role("fileno", roles.categorical)

print(population)

Connected to project 'my_project'.

getML Engine is already running.


/usr/local/lib/python3.11/dist-packages/getml/data/columns/columns.py:1833: UserWarning: After parsing time stamps, column 'Uhrzeit' likely contains only nan values.

Check the supplied 'time_formats' and the data.
  warnings.warn(


  name                       Uhrzeit         id   fileno             I_In        V_In    I_Out(A)    V_Out(V)    Temp(°C)
  role                    time_stamp   join_key   categorical   numerical   numerical   numerical   numerical   numerical
  unit   time stamp, comparison only                                                                                     
     0                          NULL          0   0                  0.25       11.65        0.31        9.03       37.38
     1                          NULL          1   0                  0.27       11.7         0.27        9          37.38
     2                          NULL          2   0                  0.44       11.74        0.27        9          37.38
     3                          NULL          3   0                  0.31       11.61        0.41        9          37.38
     4                          NULL          4   0                  0.31       11.7         0.33        9          37.63
                        